# Objetivos deste trabalho
- Familiarizar-se com a biblioteca PyTorch
- Definir arquiteturas MLP simples em PyTorch
- Treinar utilizando CIFAR10, testando diferentes arquiteturas, parâmetros, funções de loss e otimizadores
- Comparar os resultados obtidos utilizando apenas Perpceptrons

## Inicialização 

In [26]:
%matplotlib inline

import time

import numpy as np 
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as transforms

In [27]:
# Carregar os datasets

transform=transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor()
])

dataset_train = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

dataset_test = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [28]:
train_loader = DataLoader(dataset=dataset_train, batch_size=50, 
                          shuffle=True, num_workers=2)
test_loader = DataLoader(dataset=dataset_test, batch_size=50, 
                         shuffle=False, num_workers=2)
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Definindo as Arquiteturas MLP

In [29]:
# Arquitetura MLP com três Camadas Ocultas
class MLP(nn.Module):
    
    # Construtor da Classe MLP
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(32*32, 512)
        self.fc2 = nn.Linear(512,   256)
        self.fc3 = nn.Linear(256,   128)
        self.fc4 = nn.Linear(128,    10)
        self.activation_function = nn.ReLU()
    
    # Função Forward
    def forward(self, x):
        x = x.view(-1, 32*32)
        x = self.activation_function(self.fc1(x))
        x = self.activation_function(self.fc2(x))
        x = self.activation_function(self.fc3(x))
        x = self.activation_function(self.fc4(x))
        return x

### Verificação se é possivel utilizar _cuda_

In [30]:
# Confere se pode utilizar cuda
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Imprime o dispotivo
print(device)

cpu


### Impressão dos modelos

In [38]:
# Vai ser necessário 9 modelos, pois temos 3 Otimizadores e 3 funções de Loss
# e será feito todas as combinações possiveis

# Inicializa os 9 modelos
models = []
for i in range(9):
    model = MLP()
    models.append(model.to(device))

models_types = ('SGD + L1', 'SGD + MSE', 'SGD + CrossEntropy',
                'ADAM + L1', 'ADAM + MSE', 'ADAM + CrossEntropy',
                'RMSprop + L1', 'RMSprop + MSE', 'RMSprop + CrossEntropy')    

# Imprime os modelos
#print(models)
print(models_types)

('SGD + L1', 'SGD + MSE', 'SGD + CrossEntropy', 'ADAM + L1', 'ADAM + MSE', 'ADAM + CrossEntropy', 'RMSprop + L1', 'RMSprop + MSE', 'RMSprop + CrossEntropy')


# Definindo os Otimizadores e as Funções de Loss para cada Modelo

In [34]:

# Otimizadores para o Modelo de 1 camada oculta

# Inicialização
i = 0
optimizers = []

# Os três primeiros utilizam SGD
for _ in range(3):
    optimizers.append(torch.optim.SGD(models[i].parameters(), lr=0.001, momentum=0.9))
    i += 1

# Os três seguintes utilizam ADAM
for _ in range(3):
    optimizers.append(torch.optim.Adam(models[i].parameters(), lr=0.001))
    i += 1

# Os três ultimos utilizam RMSprop
for _ in range(3):
    optimizers.append(torch.optim.RMSprop(models[i].parameters(), lr=0.001, momentum=0.9))
    i += 1
    
#print(models_types)
#print(optimizers)

# Funções de Loss
loss_L1 = nn.L1Loss()
loss_MSE = nn.MSELoss()
loss_CE = nn.CrossEntropyLoss()

# Definindo a função de Treinamento

In [35]:
# Função de Treinamento 
def train(dataset, model, optimizer, loss_fn, epochs=20):
    
    # Vetor de Loss Inicializado
    loss_array = []
    
    # Percore pelo número de épocas
    for epoch in range(epochs):
        
        # Inicializa o running_loss em 0
        running_loss = 0.0
        
        # Percorre todo o dataset
        for i, data in enumerate(dataset, 0):

            # Pega a Entrada e o Label
            input, label = data
            
            # Transforma para o device
            input, label = input.to(device), label.to(device)

            # Zera os parâmetros dos gradientes
            optimizer.zero_grad()
            
            # Passa as entradas pelo modelo
            output = model(input)
            
            # Calcula o loss com a função escolhida
            loss = loss_fn(output, label)
            
            # Propaga o erro
            loss.backward()
            
            # Atualiza os pesos
            optimizer.step()
            
            # Soma ao loss
            running_loss += loss.item()
        
        # Imprime o loss desta época
        loss_array.append(running_loss/len(dataset))
        print('[%3d] loss: %.4f' %
             (epoch + 1, loss_array[-1]))
    
    # Retorna o array com os Loss em cada época
    return loss_array


# Definindo a Função de Avalaliação do Dataset por um modelo especifico

In [36]:

# Função de avaliação do modelo num conjunto passado
def evaluate(dataset, model, type_of_images='test'):
    
    # Inicialização
    correct, total = 0, 0
    
    # Percorre todo o dataset
    with torch.no_grad():
        for data in dataset:
            
            # Transforma para o device
            image, label = data
            image, label = image.to(device), label.to(device)
            
            # Recebe a saida do modelo
            output = model(image)
            
            # Faz o resultado dos labels das saidas
            _, predicted = torch.max(output.data, 1)
            
            # Confere se o resultado foi igual o label para dar acerto
            total += 1 
            correct += (predicted == label).sum().item()
    
    # Imprime a acurácia do dataset para tal modelo
    print('Accuracy of the network on the %d %s images: %d %%' % 
          ( len(test_loader),  type_of_images, 100 * correct / total))
    
    # Retorna a acurácia
    return (100 * correct / total)


# __TREINAMENTO DOS MODELOS__

In [37]:
print(models_types)
loss_array = []
acuracies = []

('SGD + L1', 'SGD + MSE', 'SGD + CrossEntropy', 'ADAM + L1', 'ADAM + MSE', 'ADAM + CrossEntropy', 'RMSprop + L1', 'RMSprop + L1', 'RMSprop + CrossEntropy')


### Utilizando SGD + L1

In [ ]:
loss_array.append(train(train_loader, models[0], optimizers[0], loss_L1))

### Utilizando SGD + MSE

In [ ]:
loss_array.append(train(train_loader, models[1], optimizers[1], loss_MSE))

### Utilizando SGD + CrossEntropy

In [ ]:
loss_array.append(train(train_loader, models[2], optimizers[2], loss_CE))

### Utilizando ADAM + L1

In [ ]:
loss_array.append(train(train_loader, models[3], optimizers[3], loss_L1))

### Utilizando ADAM + MSE

In [ ]:
loss_array.append(train(train_loader, models[4], optimizers[4], loss_MSE))

### Utilizando ADAM + CrossEntropy

In [ ]:
loss_array.append(train(train_loader, models[5], optimizers[5], loss_CE))

### Utilizado RMSprop + L1

In [ ]:
loss_array.append(train(train_loader, models[6], optimizers[6], loss_L1))

### Utilizado RMSprop + MSE

In [ ]:
loss_array.append(train(train_loader, models[7], optimizers[7], loss_MSE))

### Utilizado RMSprop + CrossEntropy

In [ ]:
loss_array.append(train(train_loader, models[8], optimizers[8], loss_CE))

## Gráficos da Loss em Cada Época para todos os modelos

In [ ]:
# Passa por Todos os modelos
for i in range(9):
    # Imprime o Gráfico de Desempenho de Loss
    plt.title(models_types[i])
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.plot(loss_array[i], 'b-')
    plt.show()
    
    # Imprime a acurácia do Dataset de Teste
    acuracies.append(evaluate(test_loader, models[i]))

## Gráficos de Barras diferenciando a acurácia entre cada modelo

In [ ]:

x = np.arange(9)
plt.bar(x, acuracies)
plt.xticks(x, models_types)
plt.show()